# Inferring
In this lesson, you will infer sentiment and topics from product reviews and news articles.

## Setup

In [2]:
import openai
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.getenv('OPENAI_API_KEY')

In [3]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

## Product review text

In [4]:
lamp_review = """
Needed a nice lamp for my bedroom, and this one had \
additional storage and not too high of a price point. \
Got it fast.  The string to our lamp broke during the \
transit and the company happily sent over a new one. \
Came within a few days as well. It was easy to put \
together.  I had a missing part, so I contacted their \
support and they very quickly got me the missing piece! \
Lumina seems to me to be a great company that cares \
about their customers and products!!
"""

## Sentiment (positive/negative)

In [5]:
prompt = f"""
What is the sentiment of the following product review, 
which is delimited with triple backticks?

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

The sentiment of the product review is positive.


In [6]:
prompt = f"""
What is the sentiment of the following product review, 
which is delimited with triple backticks?

Give your answer as a single word, either "positive" \
or "negative".

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

positive


In [10]:
lamp_review2 = """
Needed a nice lamp for my bedroom, and this one had \
additional storage and not too high of a price point. \
Got it fast.  The string to our lamp broke during the \
transit and the company ignored us. \
I can't believe i spent the money on this product!! \
The customer service was terrible, the quality of the product \
is terrible, i can't see in my bedroom, and i'm left with \
having to go shopping for another lamp. Why can't they stand \
by their products?
"""

In [11]:
prompt = f"""
What is the sentiment of the following product review, 
which is delimited with triple backticks?

Review text: '''{lamp_review2}'''
"""
response = get_completion(prompt)
print(response)

The sentiment of the product review is negative.


In [12]:
prompt = f"""
Is the writer of the following review expressing anger?\
The review is delimited with triple backticks. \
Give your answer as either yes or no.

Review text: '''{lamp_review2}'''
"""
response = get_completion(prompt)
print(response)

Yes


## Identify types of emotions

In [7]:
prompt = f"""
Identify a list of emotions that the writer of the \
following review is expressing. Include no more than \
five items in the list. Format your answer as a list of \
lower-case words separated by commas.

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

satisfied, pleased, grateful, impressed, happy


## Identify anger

In [8]:
prompt = f"""
Is the writer of the following review expressing anger?\
The review is delimited with triple backticks. \
Give your answer as either yes or no.

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

No


## Extract product and company name from customer reviews

In [13]:
prompt = f"""
Identify the following items from the review text: 
- Item purchased by reviewer
- Company that made the item

The review is delimited with triple backticks. \
Format your response as a JSON object with \
"Item" and "Brand" as the keys. 
If the information isn't present, use "unknown" \
as the value.
Make your response as short as possible.
  
Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

{
  "Item": "lamp",
  "Brand": "Lumina"
}


## Doing multiple tasks at once

In [17]:
prompt = f"""
Identify the following items from the review text: 
- Sentiment (positive or negative)
- Is the reviewer expressing anger? (true or false)
- Item purchased by reviewer
- Company that made the item

The review is delimited with triple backticks. \
Format your response as a JSON object with \
"Sentiment", "Anger", "Item" and "Brand" as the keys.
If the information isn't present, use "unknown" \
as the value.
Make your response as short as possible.
Format the Anger value as a boolean.

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)



{
  "Sentiment": "positive",
  "Anger": false,
  "Item": "lamp",
  "Brand": "Lumina"
}


## Inferring topics

In [18]:
story = """
In a recent survey conducted by the government, 
public sector employees were asked to rate their level 
of satisfaction with the department they work at. 
The results revealed that NASA was the most popular 
department with a satisfaction rating of 95%.

One NASA employee, John Smith, commented on the findings, 
stating, "I'm not surprised that NASA came out on top. 
It's a great place to work with amazing people and 
incredible opportunities. I'm proud to be a part of 
such an innovative organization."

The results were also welcomed by NASA's management team, 
with Director Tom Johnson stating, "We are thrilled to 
hear that our employees are satisfied with their work at NASA. 
We have a talented and dedicated team who work tirelessly 
to achieve our goals, and it's fantastic to see that their 
hard work is paying off."

The survey also revealed that the 
Social Security Administration had the lowest satisfaction 
rating, with only 45% of employees indicating they were 
satisfied with their job. The government has pledged to 
address the concerns raised by employees in the survey and 
work towards improving job satisfaction across all departments.
"""

## Infer 5 topics

In [19]:
prompt = f"""
Determine five topics that are being discussed in the \
following text, which is delimited by triple backticks.

Make each item one or two words long. 

Format your response as a list of items separated by commas.

Text sample: '''{story}'''
"""
response = get_completion(prompt)
print(response)

1. Government survey
2. Department satisfaction rating
3. NASA
4. Social Security Administration
5. Job satisfaction improvement


In [20]:
response.split(sep=',')

['1. Government survey\n2. Department satisfaction rating\n3. NASA\n4. Social Security Administration\n5. Job satisfaction improvement']

In [21]:
topic_list = [
    "nasa", "local government", "engineering", 
    "employee satisfaction", "federal government"
]

## Make a news alert for certain topics

In [25]:
prompt = f"""
Determine whether each item in the following list of \
topics is a topic in the text below, which
is delimited with triple backticks.

Give your answer as list with 0 or 1 for each topic.\

List of topics: {", ".join(topic_list)}

Output the results in JSON format, with the topic as the element \
and your answer as the value.

Text sample: '''{story}'''
"""
response = get_completion(prompt)
print(response)

{
  "nasa": 1,
  "local government": 0,
  "engineering": 0,
  "employee satisfaction": 1,
  "federal government": 1
}


In [27]:
#topic_dict = {i.split(': ')[0]: int(i.split(': ')[1]) for i in response.split(sep='\n')}
import json
jsonData = json.loads(response)
if jsonData['nasa'] == 1:
    print("ALERT: New NASA story!")

ALERT: New NASA story!


## Try experimenting on your own!

In [35]:
story = """
SpaceX recently made headlines as it became the second most valuable private company — worth a reported $150 billion — behind only TikTok parent ByteDance. But not all space tech companies have seen the valuation and fundraising success of Elon Musk‘s aviation and aerospace darling.

Funding to VC-backed space tech startups — defined here as space travel, satellite communication and aerospace — has dramatically decreased in the last year-plus, and this year is on pace for the lowest total dollars since 2020, according to Crunchbase data

Like most sectors, space tech funding rocketed to the stars in 2021, hitting an all-time high of $12.1 billion in more than 450 deals. The sector only saw about a 25% decline last year, when space tech startups raised $9 billion in just under 400 deals.

However, this year has been a different story. With the year more than half over, the sector has witnessed only $2.4 billion of funding in just 172 deals.

Big round slowdown
One of the main reasons funding has declined is the disappearance of the huge rounds the sector has become so associated with thanks to the likes of SpaceX and Relativity Space.

In 2021, space tech startups received 14 rounds of a quarter-billion dollars or more, including Louisville, Colorado-based Sierra Space raising a $1.4 billion round and SpaceX locking up an $850 million round.

Last year, the number of those types of rounds dropped to only five, but did include a $1.7 billion raise by SpaceX and Costa Mesa, California-based defense firm Anduril closing a $1.5 billion Series E that valued the company at $8.5 billion.

This year, however, no space startup has come that close to such a round. Some of the biggest rounds this year include:

In March, Germany-based Isar Aerospace, which helps launch small and medium satellites, raised a Series C of just more than $174 million.
In early April, San Francisco-based satellite developer Astranis locked up a $200 million round at a $1.6 billion valuation, per a Bloomberg report. The new deal was led by Andreessen Horowitz and came just before the company launched its first satellite aboard the SpaceX Falcon 9 rocket.
In late April, Berthoud, Colorado-based Ursa Major closed on a $100 million investment. BlackRock and Space Capital took part in the funding. The company is developing the next generation of rocket engines with substantially more power. According to a report, Ursa Major was looking to secure the funding at a $400 million pre-money valuation.

Of course, big rounds are down in mostly all sectors as VCs and growth firms have pulled back significantly in funding such rounds at big valuations.

The numbers do not mean the “Space 2.0” era — the current wave of investment that came after the initial excitement of firms like SpaceX and Rocket Lab being founded — is over.

Space tech — like other deep sectors — typically sees the most dramatic pullback from investors in a slowing market as they look for more “sure” bets with a much shorter time horizon to liquidity.

The near collapse of the SPAC market also likely has caused investor anxiety, as one avenue for potential exits has been severely limited. It was just a couple of years ago when startups including Google-backed Planet Labs and Spire Global were able to access the public market through such vehicles, but the poor performance of SPACs in general has made that a less than likely option for many companies.

However, there is no denying the interest in new launch and propulsion technologies, as well as other opportunities space may hold — think manufacturing or mining. Such markets and the possible return on investment from such space opportunities are likely too much for VCs to resist.

While the days of spending like in 2021 may be gone, it is unlikely investors are going to stop looking to go where no man (or woman) has gone before.
"""


topic_list = [
    "nasa", "local government", "engineering", 
    "employee satisfaction", "federal government", "spacex",
    "venture", "funding", "space", 
]


prompt = f"""
Perform the following actions:
1 - Using no more than 3 sentences, summarize the following text delimited with triple backticks.
2 -  Determine whether each item in the following list of \
topics is a topic in the text below, which
is delimited with triple backticks.

Give your answer as list with 0 or 1 for each topic.\

List of topics: {", ".join(topic_list)}

3 - Output the results in JSON format. Use  summary for your output in step 1. \
    Use topics as as a JSON object element containing the list of topics and their values.

Your response should only include the JSON.

Text sample: '''{story}'''
"""
response = get_completion(prompt)
print(response)


import json
jsonData = json.loads(response)
if jsonData['topics']['spacex'] == 1 & jsonData['topics']['funding'] == 1:
    print("ALERT: New SpaceX Funding story!")


{
  "summary": "SpaceX recently made headlines as it became the second most valuable private company — worth a reported $150 billion — behind only TikTok parent ByteDance. But not all space tech companies have seen the valuation and fundraising success of Elon Musk‘s aviation and aerospace darling.",
  "topics": {
    "nasa": 0,
    "local government": 0,
    "engineering": 0,
    "employee satisfaction": 0,
    "federal government": 0,
    "spacex": 1,
    "venture": 0,
    "funding": 1,
    "space": 1
  }
}
ALERT: New SpaceX Funding story!
